In [192]:
import pandas as pd
import numpy as np

import requests

import os
import time
from datetime import datetime
import sys
import yaml
import gc


from data_loader import *
from utils import *
from nn import *
from models import *
from llm import *


import seaborn as sns
sns.set_style("darkgrid")
plt.rcParams.update({
    'axes.facecolor': '#1e1e1e',
    'figure.facecolor': '#1e1e1e',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'axes.grid': True,
    'grid.color': 'gray'
})

pd.set_option("display.max_column",None)
print(os.getcwd())


%load_ext autoreload
%reload_ext autoreload
%autoreload 2


def auto_reload():
    %load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
with open("config.yaml",'r') as f:
    config = yaml.safe_load(f)

home_dir = config['HOME_DIRECTORY']
home_dir

'/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG'

In [4]:
# Model File Configs:
# Every Run of the notebook is logged in to a submodel folder for it

############################################################################
# Please set output path to the project directory where it is uncompressed #
############################################################################

project_path = home_dir + "/outputs/models/"




output_path = ""

submodel_name = ""

encoding_path = ""
feature_report_path = ""
def create_submodel(model_name:str):
    author = "EJ"
    global submodel_name
    submodel_name = datetime.now().strftime("%d_%H_%M") + "_"+model_name
    global output_path
    output_path = project_path+submodel_name
    global encoding_path
    encoding_path = output_path+"/encodings/"
    global feature_report_path
    feature_report_path = output_path+"/feature_report/"
    os.mkdir(output_path)
    os.mkdir(encoding_path)
    os.mkdir(feature_report_path)


In [5]:
create_submodel("test")

# API Football:
https://www.api-football.com

![PYTHON LOGO](https://www.api-football.com/public/img/news/archi-beta.jpg)

In [6]:
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']]

,league_id,league_name,country_name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil
...,...,...,...
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World
1130,1133,Goiano - 3,Brazil
1131,1001,CONCACAF Women U20,World


# Leagues subset:

In [7]:
# Configs
major_leagues = ["Premier League","La Liga","Serie A","Bundesliga","Eredivisie","Ligue 1"]
major_countries = ["England","Spain","Italy","Germany","Netherlands","France","Brazil"]
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = [2022,2021,2023,2024] # seasons to pull players and teams stats of



leagues_subset = leagues_dat[leagues_dat.league_name.isin(major_leagues) & leagues_dat.country_name.isin(major_countries)] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [8]:
leagues_subset

,seasons,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag
2,"[{'coverage': {'fixtures': {'events': True, 'l...",61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg
4,"[{'coverage': {'fixtures': {'events': True, 'l...",71,Serie A,League,https://media.api-sports.io/football/leagues/7...,Brazil,BR,https://media.api-sports.io/flags/br.svg
5,"[{'coverage': {'fixtures': {'events': True, 'l...",39,Premier League,League,https://media.api-sports.io/football/leagues/3...,England,GB,https://media.api-sports.io/flags/gb.svg
6,"[{'coverage': {'fixtures': {'events': True, 'l...",78,Bundesliga,League,https://media.api-sports.io/football/leagues/7...,Germany,DE,https://media.api-sports.io/flags/de.svg
7,"[{'coverage': {'fixtures': {'events': True, 'l...",135,Serie A,League,https://media.api-sports.io/football/leagues/1...,Italy,IT,https://media.api-sports.io/flags/it.svg
8,"[{'coverage': {'fixtures': {'events': True, 'l...",88,Eredivisie,League,https://media.api-sports.io/football/leagues/8...,Netherlands,NL,https://media.api-sports.io/flags/nl.svg
10,"[{'coverage': {'fixtures': {'events': True, 'l...",140,La Liga,League,https://media.api-sports.io/football/leagues/1...,Spain,ES,https://media.api-sports.io/flags/es.svg


# Read All fixtures data

In [9]:
teams_dat = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [10]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_30482/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_30482/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_30482/1208416677.py:6: FutureWarning: The behavior of DataFrame concatena

In [11]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")
complete_data['fixture_date'] = pd.to_datetime(complete_data.fixture_date)
complete_data = create_datetime_columns(complete_data,'fixture_date')

# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:47: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['month_e'] = data[dt_col].dt.to_period('M')
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:51: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['week_e'] = data[dt_col].dt.to_period('W-MON')


In [12]:
complete_data[complete_data.fixture_date.notna()].shape

(307241, 59)

In [14]:
complete_data.head()

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accuracy,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,opponent,team_winner,team,fixture_date,outcome,dribble_success_rate,target_shot_conversion_perc,duels_won_perc,pass_accuracy_perc,year_e,month_e,month_name_e,day_of_week_e,day_of_week_name_e,week_e,outcome_num,win
0,47269,Álex Remiro,NaN,90.0,1,G,6.2,False,False,NaN,NaN,NaN,2,0.0,2.0,31.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0.0,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,2025-05-24 14:15:00+00:00,loss,NaN,NaN,NaN,83.870968,2025.0,2025-05,May,5.0,Saturday,2025-05-20,1,0
1,2202,Hamari Traoré,NaN,75.0,18,D,6.9,False,False,NaN,NaN,NaN,0,0.0,NaN,37.0,NaN,30.0,2.0,NaN,NaN,8.0,5.0,1.0,1.0,1.0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,2025-05-24 14:15:00+00:00,loss,100.0,NaN,62.500000,81.081081,2025.0,2025-05,May,5.0,Saturday,2025-05-20,1,0
2,47298,Aritz Elustondo,NaN,90.0,6,D,6.9,False,False,1.0,NaN,NaN,0,0.0,NaN,77.0,1.0,70.0,1.0,1.0,1.0,10.0,6.0,NaN,NaN,NaN,4.0,2.0,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,2025-05-24 14:15:00+00:00,loss,NaN,NaN,60.000000,90.909091,2025.0,2025-05,May,5.0,Saturday,2025-05-20,1,0
3,182546,Jon Pacheco,NaN,90.0,20,D,7.5,False,False,NaN,NaN,NaN,0,0.0,NaN,51.0,NaN,45.0,1.0,1.0,1.0,6.0,4.0,NaN,NaN,NaN,1.0,1.0,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,2025-05-24 14:15:00+00:00,loss,NaN,NaN,66.666667,88.235294,2025.0,2025-05,May,5.0,Saturday,2025-05-20,1,0
4,182636,Javi López,NaN,90.0,12,D,5.9,False,False,1.0,NaN,NaN,0,0.0,NaN,39.0,NaN,34.0,NaN,1.0,NaN,5.0,1.0,1.0,NaN,1.0,1.0,NaN,0,0,NaN,NaN,0,0,NaN,1208500,1,1.0,0,2,2.0,0,Rayo Vallecano,Rayo Vallecano,Real Sociedad,2025-05-24 14:15:00+00:00,loss,NaN,NaN,20.000000,87.179487,2025.0,2025-05,May,5.0,Saturday,2025-05-20,1,0


In [12]:
complete_data.columns

Index(['player_id', 'player_name', 'offsides', 'games_minutes', 'games_number',
       'games_position', 'games_rating', 'games_captain', 'games_substitute',
       'shots_total', 'shots_on', 'goals_total', 'goals_conceded',
       'goals_assists', 'goals_saves', 'passes_total', 'passes_key',
       'passes_accuracy', 'tackles_total', 'tackles_blocks',
       'tackles_interceptions', 'duels_total', 'duels_won',
       'dribbles_attempts', 'dribbles_success', 'dribbles_past', 'fouls_drawn',
       'fouls_committed', 'cards_yellow', 'cards_red', 'penalty_won',
       'penalty_commited', 'penalty_scored', 'penalty_missed', 'penalty_saved',
       'fixture_id', 'team_goals_scored', 'team_non_penalty_goals_scored',
       'team_goals_scored_half', 'team_goals_conceded',
       'team_non_penalty_goals_conceded', 'team_goals_conceded_half',
       'opponent', 'team_winner', 'team', 'fixture_date', 'outcome',
       'dribble_success_rate', 'target_shot_conversion_perc', 'duels_won_perc',
     

In [31]:
question ="What is the latest date that you can answer from?"
answer_dat =  ask_gpt_q(question)

print(f"question: {question}")
print("")
print(answer_dat['answer'].values[0])
print("")
print(f"Answer Certainty: {answer_dat['certainty']}")

question: What is the latest date that you can answer from?

The latest date I can answer from is October 2023, as my knowledge cutoff is set to that date.

Answer Certainty: 0    0.95
Name: certainty, dtype: float64


In [13]:
liv_dat = complete_data[complete_data.team == 'Liverpool']

In [ ]:
liv_dat.groupby(['fixture_id']).agg(avg_key_pass_forwards = ("passes_key",lambda x : liv_dat[]))

In [16]:
# This is the dictionary that contains all information about the features    
dat_dict = find_data_types(complete_data,config['OUTCOME_COLS'] + ['outcome_num','outcome'])
dat_dict = pd.DataFrame(list(dat_dict.items()),columns =['feature','type'])

# differentiate modeling features
non_modeling_features = config['FIXTURE_COLS'] + config['OUTCOME_COLS'] + config['MISC_COLS'] + ['outcome_num','win','fixture_date','major_position']
dat_dict['modeling_feature'] = np.where(dat_dict['feature'].isin(non_modeling_features),0,1)
dat_dict['encoded'] = 0

print(dat_dict['type'].value_counts())
dat_dict.reset_index(drop= True)

## Encode Features
dat_dict = create_data_index(complete_data,dat_dict,'target',encoding_path)
dat_dict[dat_dict.modeling_feature ==1]

type
numeric        38
target         10
categorical     6
datetime        3
other           2
Name: count, dtype: int64


,feature,type,modeling_feature,encoded
2,offsides,numeric,1,0
9,shots_total,numeric,1,0
10,shots_on,numeric,1,0
11,goals_total,numeric,1,0
12,goals_conceded,numeric,1,0
13,goals_assists,numeric,1,0
14,goals_saves,numeric,1,0
15,passes_total,numeric,1,0
16,passes_key,numeric,1,0
17,passes_accuracy,numeric,1,0


In [46]:

gpt_test = complete_data[(complete_data.team == 'Liverpool') & (complete_data.year_e == 2025)]
gpt_test.groupby('player_name').agg(total_goals = ("goals_total","sum")).sort_values('total_goals',ascending = False)

,total_goals
player_name,
Mohamed Salah,29.0
Luis Díaz,13.0
Cody Gakpo,10.0
Diogo Jota,6.0
Dominik Szoboszlai,6.0
Alexis Mac Allister,5.0
Darwin Núñez,5.0
Virgil van Dijk,3.0
Trent Alexander-Arnold,3.0


In [35]:
gpt_answer = ask_gpt(complete_data[complete_data.player_name.str.contains("Giovanni Leoni")],"give me a summary for this player")

Raw GPT JSON: {'player_id': 409047, 'player_name': 'Giovanni Leoni', 'total_matches': 39, 'average_minutes_played': 67.7, 'average_rating': 6.7, 'goals_scored': 0, 'assists': 0, 'shots_total': 0, 'shots_on_target': 0, 'total_passes': 759, 'average_pass_accuracy': 87.9, 'tackles': 50, 'duels_won': 44, 'duels_attempted': 56, 'duels_success_rate': 78.57, 'dribbles_attempted': 12, 'dribbles_successful': 10, 'dribble_success_rate': 83.33, 'yellow_cards': 4, 'red_cards': 1, 'penalty_scored': 0, 'outcomes': {'win': 25.6, 'draw': 28.2, 'loss': 46.2}, 'performance_highlight': 'Player consistently performed as a defender without significant offensive contributions but managed strong passing and defensive skills with good duel rates.'}


In [38]:
gpt_answer.drop_duplicates()

,player_id,player_name,total_matches,average_minutes_played,average_rating,goals_scored,assists,shots_total,shots_on_target,total_passes,average_pass_accuracy,tackles,duels_won,duels_attempted,duels_success_rate,dribbles_attempted,dribbles_successful,dribble_success_rate,yellow_cards,red_cards,penalty_scored,outcomes,performance_highlight
win,409047,Giovanni Leoni,39,67.7,6.7,0,0,0,0,759,87.9,50,44,56,78.57,12,10,83.33,4,1,0,25.6,Player consistently performed as a defender wi...
draw,409047,Giovanni Leoni,39,67.7,6.7,0,0,0,0,759,87.9,50,44,56,78.57,12,10,83.33,4,1,0,28.2,Player consistently performed as a defender wi...
loss,409047,Giovanni Leoni,39,67.7,6.7,0,0,0,0,759,87.9,50,44,56,78.57,12,10,83.33,4,1,0,46.2,Player consistently performed as a defender wi...


In [63]:
complete_data[complete_data.player_name.str.contains("Giovanni Leoni")].groupby('player_name').agg(duels_won = ("duels_won","sum"),
                                                                                                   duels_attempts = ("duels_total","sum"),
                                                                                                   duels_won_perc = ("duels_won_perc","mean"),
                                                                                                   tackles = ("tackles_total","sum"),
                                                                                                   blocks = ("tackles_blocks","sum"),
                                                                                                   interceptions = ("tackles_interceptions","sum"),
                                                                                                   avg_rating = ("games_rating",lambda x: x.astype("float64").mean() ),
                                                                                                   )

,duels_won,duels_attempts,duels_won_perc,tackles,blocks,interceptions,avg_rating
player_name,,,,,,,
Giovanni Leoni,43.0,71.0,62.15873,14.0,11.0,15.0,6.652941


In [194]:
leoni = ask_gpt_strict(complete_data[complete_data.player_name.str.contains("Giovanni Leoni")],
                       "give me a comprehensive summary for this player")

In [195]:
leoni

,year_e,player_id,player_name,total_games_played,total_minutes_played,average_rating,captain_matches,substitute_appearances,total_shots,shots_on_target,goals_scored,assists,yellow_cards,red_cards,fouls_drawn,fouls_committed,attempted_dribbles,successful_dribbles,dribbled_past,dribble_success_rate,total_passes,key_passes,average_pass_accuracy,total_tackles,blocks,interceptions,duels_contested,duels_won,duels_won_percentage,penalties_won,penalties_committed,penalties_scored,penalties_missed,penalties_saved,team_goals_scored,team_non_penalty_goals,team_goals_conceded,team_non_penalty_goals_conceded,matches_won
0,2025.0,409047,Giovanni Leoni,34,1203.0,6.652941,0,20,2.0,1.0,1.0,0.0,3,1,9.0,11.0,3.0,3.0,5.0,100.0,619.0,1.0,87.07317,14.0,11.0,15.0,71.0,43.0,62.15873,0.0,0.0,0,0,0.0,36,36.0,54,54.0,4


In [196]:
van_dijk = ask_gpt_strict(complete_data[(complete_data.player_name.str.contains("Virgil van Dijk")) & (complete_data.year_e.isin([2024,2025]))],
                          "give me a comprehensive summary for this player")

In [197]:
van_dijk

,year_e,player_id,player_name,total_games_played,total_minutes_played,average_rating,captain_matches,substitute_appearances,total_shots,shots_on_target,goals_scored,assists,yellow_cards,red_cards,fouls_drawn,fouls_committed,attempted_dribbles,successful_dribbles,dribbled_past,dribble_success_rate,total_passes,key_passes,average_pass_accuracy,total_tackles,blocks,interceptions,duels_contested,duels_won,duels_won_percentage,penalties_won,penalties_committed,penalties_scored,penalties_missed,penalties_saved,team_goals_scored,team_non_penalty_goals,team_goals_conceded,team_non_penalty_goals_conceded,matches_won
0,2024.0,290,Virgil van Dijk,19,1710.0,7.257895,19,0,18.0,7.0,1.0,0.0,2,0,8.0,12.0,3.0,1.0,2.0,100.0,1530.0,4.0,92.342040,15.0,18.0,15.0,114.0,82.0,72.307014,0.0,0.0,0,0,0.0,47,47.0,25,25.0,12
1,2025.0,290,Virgil van Dijk,38,3330.0,7.270270,37,1,24.0,8.0,3.0,1.0,5,0,13.0,17.0,5.0,3.0,9.0,100.0,2920.0,12.0,91.556997,38.0,16.0,56.0,252.0,173.0,68.360045,0.0,0.0,0,0,0.0,86,86.0,41,41.0,25


In [198]:
def_compare = pd.concat([van_dijk,leoni],axis = 0).T
def_compare.columns = def_compare.loc['player_name'].values
def_compare = def_compare.drop(index='player_name')
def_compare.index.name = 'stat_type'



In [199]:
def_compare

,Virgil van Dijk,Virgil van Dijk,Giovanni Leoni
stat_type,,,
year_e,2024.0,2025.0,2025.0
player_id,290,290,409047
total_games_played,19,38,34
total_minutes_played,1710.0,3330.0,1203.0
average_rating,7.257895,7.27027,6.652941
captain_matches,19,37,0
substitute_appearances,0,1,20
total_shots,18.0,24.0,2.0
shots_on_target,7.0,8.0,1.0


In [ ]:
# primary position map:
player_position = complete_data.groupby(["player_id","games_position"],as_index = False).agg(games_played = ("player_id","size"))
player_position['multiple_records'] = player_position.groupby('player_id')['games_played'].transform("cumsum")
player_position['multiple_records'] = player_position.groupby('player_id')['multiple_records'].transform("max")
player_position['major_position'] = np.where(player_position.games_played/player_position.multiple_records >= .5, player_position.games_position,None)
player_position_map = player_position[['player_id','major_position']].dropna().drop_duplicates()
player_position_map

# Join back to complete_data

complete_data = pd.merge(complete_data,player_position_map,on = 'player_id',how = 'left')

In [ ]:
# quick knn clustering
from sklearn.cluster import KMeans

# who is ekitike most similar to 

interest_col_set = list(set(['major_position','player_id','player_name'] + config["ATTACK_COLS"] +  config["PASSING_COLS"]))


# primary position:



striker_stats = complete_data[interest_col_set].fillna(0)[complete_data[interest_col_set].fillna(0).major_position.isin(['F'])].groupby(['player_id','player_name'],as_index = False).agg(n_games = ('player_name','size'),
                                                                          avg_dribble_success = ('dribble_success_rate','mean'),
                                                                          std_dribble_success = ('dribble_success_rate','std'),
                                                                          avg_target_shot_cr = ('target_shot_conversion_perc','mean'),
                                                                          std_target_shot_cr = ('target_shot_conversion_perc','std'),
                                                                          avg_duels_cr = ('duels_won_perc','mean'),
                                                                          std_duels_cr = ('duels_won_perc','std'),
                                                                          avg_pass_accuracy = ('pass_accuracy_perc','mean'),
                                                                          std_pass_accuracy = ('pass_accuracy_perc','std')
                                                                          )


striker_stats = striker_stats[striker_stats.n_games > 40]
striker_stats.shape


In [ ]:
striker_stats.sort_values('avg_dribble_success',ascending=False)

In [ ]:
stats = ['avg_target_shot_cr','avg_dribble_success','avg_duels_cr','avg_pass_accuracy']

for stat in stats:
    striker_stats = fit_optimal_kmeans(striker_stats,stat)

# Clustering

In [ ]:
# Best Goal Scorers:
striker_stats.sort_values(["avg_target_shot_cr","cluster_avg_target_shot_cr"],ascending= [0,0])[["player_name","n_games","avg_target_shot_cr","std_target_shot_cr","cluster_avg_target_shot_cr"]].head(15)

In [ ]:
striker_stats[striker_stats.cluster_avg_target_shot_cr == 9]

In [ ]:
# Best Dribblers:
striker_stats.sort_values(["avg_dribble_success","cluster_avg_dribble_success"],ascending= [0,0])[["player_name","n_games","avg_dribble_success","std_dribble_success","cluster_avg_dribble_success"]].head(20)

In [ ]:
striker_stats[striker_stats.cluster_avg_dribble_success == 20]

In [ ]:
striker_stats.sort_values(["avg_duels_cr","cluster_avg_duels_cr"],ascending= [0,0])[["player_name","n_games","avg_duels_cr","std_duels_cr","cluster_avg_duels_cr"]].head(20)

In [ ]:
striker_stats.sort_values(["avg_pass_accuracy","cluster_avg_pass_accuracy"],ascending= [0,0])[["player_name","n_games","avg_pass_accuracy","std_pass_accuracy","cluster_avg_pass_accuracy"]].head(15)

In [ ]:
striker_stats[striker_stats.cluster_pass == 4]

In [ ]:
filter_query = 'major_position.isin(["M"])'


# Stat to look at:
stat = 'target_shot_conversion_perc'
agg_fun = "mean"
rank_cutoff = 20

# configs 
min_appearance = 40

dribble_dat_g = complete_data.query(filter_query).reset_index().fillna(0).groupby("player_name").agg(n_apps = ("player_name","size"),stat = (stat,agg_fun)).reset_index()
dribble_dat_g = dribble_dat_g[dribble_dat_g.n_apps >= min_appearance]
dribble_dat_g['rank'] = dribble_dat_g["stat"].fillna(0).rank(ascending= False,method = 'dense')
dribble_dat_g.sort_values("rank",inplace = True)

fig, ax = plt.subplots(figsize=(13, 8))

# Plot correctly, no comma here
sns.boxplot(
    data=complete_data.query(filter_query)[complete_data.query(filter_query).player_name.isin(dribble_dat_g[dribble_dat_g['rank'] < rank_cutoff]['player_name'])],
    x="player_name",
    y=stat,
    order=dribble_dat_g[dribble_dat_g['rank'] < rank_cutoff]['player_name'],
    ax=ax,
    
)

# Now this works correctly on `ax`
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(f"Stat: {stat}")
plt.tight_layout()
plt.show()


In [ ]:
fig = plt.subplots(nrows=1, ncols = 1, figsize = (20,10))
fig = sns.heatmap(complete_data.query(filter_query)[config['PASSING_COLS']  + ['team_goals_scored','team_non_penalty_goals_scored','team_goals_conceded']].corr(),cmap = 'coolwarm')
fig.set_xticklabels(fig.get_xticklabels(),rotation = 60)


In [ ]:
sns.pairplot(complete_data.query(filter_query)[config['PASSING_COLS']  + ['team_goals_scored','team_non_penalty_goals_scored','team_goals_conceded']])

In [ ]:
complete_data['win'].value_counts(normalize=True)

In [ ]:
complete_data.head()

In [ ]:
# trial multiclass model:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(complete_data.query(filter_query)[list(set(config['PASSING_COLS'] + config['ATTACK_COLS'] ))],
                                                    complete_data.query(filter_query)['win'],
                                                    stratify=complete_data.query(filter_query)['win'],
                                                    random_state=33)


In [ ]:
create_submodel("catboost")

In [ ]:
output_path

In [ ]:
model = run_model_with_fs_tune(X_train, X_test, y_train, y_test,dat_dict,'catboost',output_path=output_path)

In [ ]:
dat = NNDataFromPd(X_train.fillna(0), y_train.outcome_num, dat_dict)
train_loader = DataLoader(dat, batch_size = 128,shuffle= True)

In [ ]:
train_loader.dataset.X_numeric_tensor.shape

In [ ]:
# model params
n_features = X_train.shape[1]
n_classes = y_train.iloc[:,0].nunique()
model = MultiClassModel(n_features,n_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
epochs = 500

for epoch in range(epochs):
    
    epoch_loss = 0

    for X_numeric_batch, X_categoric_batch, y_batch in train_loader:
        
        pred = model.forward(X_numeric_batch)
        
        loss = criterion(pred,y_batch)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {epoch_loss}")

In [ ]:
model.eval()

In [ ]:
X_test['passes_accuracy'] = X_test['passes_accuracy'].astype("float64")

In [ ]:
# test sets

test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

In [ ]:
model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_class = torch.argmax(output, dim = 1)

In [ ]:
# Logistic Model:
X_train, X_test, y_train, y_test = train_test_split(complete_data[complete_data.games_position == 'F'][list(set(config['DEFENSE_COLS'] + config['PASSING_COLS'] + config['ATTACK_COLS'])) + ['win']].drop(columns = 'win'),
                                                    complete_data[complete_data.games_position == 'F']['win'],
                                                    stratify=complete_data['win'],
                                                    random_state=33)

In [ ]:
train_dat = NNDataFromPd(X_train,y_train,dat_dict)
train_loader = DataLoader(train_dat,batch_size= 128,shuffle = True)

In [ ]:
n_features = X_train.shape[1]
model = LogisticNNModelComplex(n_features)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.005)

In [ ]:
epochs = 500
for epoch in range(epochs):
    epoch_loss = 0
    
    for X_numeric, X_categoric, y in train_loader:

        pred = model(X_numeric)

        loss = criterion(pred,y.unsqueeze(1))

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch},  Loss: {loss}")


In [ ]:
pred_proba.squeeze(1)

In [ ]:
from validations import *


test_dat = NNDataFromPd(X_test,y_test,dat_dict)
test_loader = DataLoader(test_dat,batch_size= X_test.shape[0],shuffle=True)

model.eval()
with torch.no_grad():
    for X_numeric_batch, X_categoric_batch, y_batch in test_loader:
        output = model(X_numeric_batch)
        pred_proba = torch.softmax(output,dim =1)
        pred_class = torch.argmax(output, dim = 1)


discrete_evaluations(y_test,pred_class,pred_proba.squeeze(1),classification_type="Binary",model_path= output_path)

In [ ]:
test_fixtures = get_team_fixtures("Liverpool",2)

In [ ]:
test_fixtures

In [ ]:
player_stat_url = "https://v3.football.api-sports.io/fixtures/players?fixture={}".format(1035045)
fixture_dat = requests.get(player_stat_url,headers=headers_api_sport)

In [ ]:
pd.json_normalize(pd.json_normalize(fixture_dat.json()['response']))['players'][0]

In [ ]:
fixture_dat_expanded = pd.concat([pd.json_normalize(pd.json_normalize(fixture_dat.json()['response'])['players'][0])[['player.id','player.name']],pd.json_normalize(pd.json_normalize(pd.json_normalize(pd.json_normalize(fixture_dat.json()['response'])['players'][0])['statistics']).rename(columns = {0:"player_stats"})['player_stats'])],axis = 1)

In [ ]:
fixtures_stat = complete_data.groupby(['fixture_id','team'],as_index=False).agg(n_opponent = ('opponent','count'),total_passes = ('passes_total','sum')).sort_values('fixture_id',ascending= False)

In [ ]:
fixtures_stat

In [ ]:
complete_data[complete_data.fixture_id == 1376437][['team','opponent']]

In [ ]:
teams_dat[teams_dat.team_name.str.contains("Tels")]

In [ ]:
angers = pd.read_parquet(home_dir + "/data/Fixtures/angers_2024.parquet")

In [ ]:
angers['fixture_date'] = pd.to_datetime(angers['fixture_date'])

In [ ]:
angers['fixture_date']